In [11]:
import sys
sys.path.append('..')
from nestedcvtraining.utils.switch_case import SwitchCase
import numpy as np
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from nestedcvtraining.utils.pipes_and_transformers import IdentityTransformer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, make_scorer, fbeta_score
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from skopt.space import Real, Integer, Categorical
from xgboost import XGBClassifier
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from nestedcvtraining.utils.pipes_and_transformers import IdentityTransformer
from nestedcvtraining.api import find_best_model, OptionedTransformer
from skopt import gbrt_minimize
from sklearn.preprocessing import PolynomialFeatures, FunctionTransformer

In [16]:
X = np.array([[1, 1, 2], [1, 2, 3], [4, 5, 6], [1, 2, 3], [4, 2, 8]])
y = np.array([1, 2, 0, 1, 2]).reshape(-1, 1)

feature_generation = SwitchCase(
    cases=[
        (
            "gen_1",
            PolynomialFeatures()
        ),
        (
            "gen_2",
            FunctionTransformer()
        )
    ],
    switch="gen_1")


preprocessor = SwitchCase(
    cases=[
        (
            "prep_1",
            Pipeline([
                ("scale", StandardScaler()), 
                ("reduce_dims", PCA(n_components=5))
            ])
        ),
        (
            "prep_2",
            Pipeline([
                ("scale", StandardScaler()), 
                ("reduce_dims", SelectKBest(mutual_info_classif, k=5)),
            ])
        )
    ],
    switch="prep_1")

model = SwitchCase(
    cases=[
        (
            "model_1",
            LogisticRegression()
        ),
        (
            "model_2",
            RandomForestClassifier()
        )
    ],
    switch="model_1")

clf = Pipeline(
    [("generator", feature_generation), ("preprocessor", preprocessor), ("model", model)]
)

In [17]:
clf

Pipeline(steps=[('generator',
                 SwitchCase(cases=[('gen_1', PolynomialFeatures()),
                                   ('gen_2', FunctionTransformer())],
                            switch='gen_1')),
                ('preprocessor',
                 SwitchCase(cases=[('prep_1',
                                    Pipeline(steps=[('scale', StandardScaler()),
                                                    ('reduce_dims',
                                                     PCA(n_components=5))])),
                                   ('prep_2',
                                    Pipeline(steps=[('scale', StandardScaler()),
                                                    ('reduce_dims',
                                                     SelectKBest(k=5,
                                                                 score_func=<function mutual_info_classif at 0x7f3b77eccb80>))]))],
                            switch='prep_1')),
                ('model',
                 SwitchCase(cases=[('model_1', LogisticRegression()),
                                   ('model_2', RandomForestClassifier())],
                            switch='model_1'))])

In [18]:
clf.get_params()

{'memory': None,
 'steps': [('generator',
   SwitchCase(cases=[('gen_1', PolynomialFeatures()),
                     ('gen_2', FunctionTransformer())],
              switch='gen_1')),
  ('preprocessor',
   SwitchCase(cases=[('prep_1',
                      Pipeline(steps=[('scale', StandardScaler()),
                                      ('reduce_dims', PCA(n_components=5))])),
                     ('prep_2',
                      Pipeline(steps=[('scale', StandardScaler()),
                                      ('reduce_dims',
                                       SelectKBest(k=5,
                                                   score_func=<function mutual_info_classif at 0x7f3b77eccb80>))]))],
              switch='prep_1')),
  ('model',
   SwitchCase(cases=[('model_1', LogisticRegression()),
                     ('model_2', RandomForestClassifier())],
              switch='model_1'))],
 'verbose': False,
 'generator': SwitchCase(cases=[('gen_1', PolynomialFeatures()),
           

In [4]:
"search_space": [
    Categorical(["gen_1", "gen_2"], name="generator__switch"),
    Categorical(["prep_1", "prep_2"], name="preprocessor__switch"),
    Categorical(["model_1", "model_2"], name="model__switch"),
    Integer(3, 10, name="preprocessor__prep_1__reduce_dims__n_components"),
    Integer(3, 10, name="preprocessor__prep_2__reduce_dims__k"),
    Integer(1, 3, name="generator__gen_1__degree"),
    Categorical(["l1", "l2"], name="model__model_1__penalty"), 
    Integer(5, 15, name="model__model_2__max_depth"),
],


SwitchCase(cases=[('option_1',
                   Pipeline(steps=[('scale', StandardScaler()),
                                   ('reduce_dims', PCA(n_components=5))])),
                  ('option_2',
                   Pipeline(steps=[('scale', StandardScaler()),
                                   ('reduce_dims',
                                    SelectKBest(k=2,
                                                score_func=<function mutual_info_classif at 0x7f69751f8a60>))])),
                  ('option_3',
                   Pipeline(steps=[('identity',
                                    <nestedcvtraining.utils.pipes_and_transformers.IdentityTransformer object at 0x7f6974747c10>)]))],
           switch='option_2')

In [5]:
sc.switch

'option_2'

In [6]:
sc.fit_transform(X, y)

/home/jaime/projects/python/nestedcvtraining/venv/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([[-0.81649658, -1.03209369],
       [-0.81649658, -0.29488391],
       [ 1.22474487,  1.91674543],
       [-0.81649658, -0.29488391],
       [ 1.22474487, -0.29488391]])

In [9]:
sc.set_params(**{
    "switch": "option_1",
    "option_1__reduce_dims__n_components": 2
})

SwitchCase(cases=[('option_1',
                   Pipeline(steps=[('scale', StandardScaler()),
                                   ('reduce_dims', PCA(n_components=2))])),
                  ('option_2',
                   Pipeline(steps=[('scale', StandardScaler()),
                                   ('reduce_dims',
                                    SelectKBest(k=2,
                                                score_func=<function mutual_info_classif at 0x7f69751f8a60>))])),
                  ('option_3',
                   Pipeline(steps=[('identity',
                                    <nestedcvtraining.utils.pipes_and_transformers.IdentityTransformer object at 0x7f6974747c10>)]))],
           switch='option_1')

In [10]:
sc.fit_transform(X, y)

array([[-1.66437265,  0.17623789],
       [-1.03343507, -0.22135093],
       [ 2.15163112, -1.02284584],
       [-1.03343507, -0.22135093],
       [ 1.57961167,  1.28930982]])

In [13]:
sc.cases[0][1].transform(X)

array([[-1.66437265,  0.17623789],
       [-1.03343507, -0.22135093],
       [ 2.15163112, -1.02284584],
       [-1.03343507, -0.22135093],
       [ 1.57961167,  1.28930982]])

In [14]:
sc.cases[1][1].transform(X)

array([[-0.81649658, -1.03209369],
       [-0.81649658, -0.29488391],
       [ 1.22474487,  1.91674543],
       [-0.81649658, -0.29488391],
       [ 1.22474487, -0.29488391]])

In [22]:
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.model_selection import StratifiedKFold


diabetes = datasets.load_diabetes()
X = diabetes.data
y = diabetes.target
lasso = linear_model.Lasso()


k_inner=5
skip_inner_folds=[2, 4]
def yield_splits(X, y, k, skip):
    cv = StratifiedKFold(n_splits=k)
    for cv_idx, (train_index, test_index) in enumerate(cv.split(X, y)):
        if k not in skip:
            yield (train_index, test_index)

scores = cross_validate(lasso, X, y, cv=yield_splits(X, y, k_inner, skip_inner_folds),
                        scoring=('r2', 'neg_mean_squared_error'),
                        return_train_score=True, return_estimator=True)
print(scores['test_neg_mean_squared_error'])

print(scores['train_r2'])

[-3367.69481873 -3489.99166356 -3926.62532842 -4576.78631641
 -4054.99821162]
[0.36792736 0.35382342 0.35195324 0.37854664 0.34732403]


/home/jaime/projects/python/nestedcvtraining/venv/lib/python3.10/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [25]:
scores["estimator"][0].predict(X)

array([184.27447089, 112.55942997, 171.6788802 , 152.60857854,
       130.07419597, 124.40085134, 116.13438608, 144.92789568,
       172.89590806, 185.52167662, 101.84131202, 133.45001925,
       132.41363209, 162.3466802 , 133.3018854 , 157.62922688,
       185.74474638, 166.9620533 , 142.32832046, 141.49625406,
       124.55199951, 124.5772406 , 134.66562185, 214.04394834,
       159.92394224, 151.12217048, 108.63482956, 171.73134247,
       141.12018488, 168.05900602, 163.30327787, 113.00127725,
       205.65447798, 127.06693957, 108.87973275, 143.77314386,
       178.48742382, 159.74485488, 204.53404317, 161.03524405,
       153.37063872, 107.77644378, 145.26072127, 129.32144004,
       189.74948166, 135.3649932 , 144.67026483, 115.74062843,
       120.25204495, 167.76875438, 144.64657032, 159.93743487,
       143.88032274, 139.93258305, 152.57146686, 113.56610215,
       177.69904896, 109.99784169, 127.2260405 , 152.66437345,
       136.12817217, 162.34811947, 126.64838326, 127.00

In [26]:
scores["estimator"][1].predict(X)

array([184.27234683, 114.4688481 , 172.24487199, 155.40691065,
       131.47037011, 126.43311326, 117.86447982, 144.66779104,
       173.30634417, 187.87395952, 104.04031225, 133.30828748,
       134.17774255, 164.44088408, 135.0242335 , 158.84073389,
       186.3334869 , 167.70012659, 144.35263893, 143.92793504,
       127.91760028, 125.2522011 , 135.27783032, 215.99638094,
       160.46034331, 153.94074577, 111.63638909, 172.17436932,
       144.36652292, 169.7070813 , 163.55308899, 116.7536236 ,
       203.65184564, 129.46231234, 111.76164849, 147.2255315 ,
       179.52734032, 162.2479056 , 203.95858093, 163.79264427,
       154.91213222, 111.81427787, 147.06538359, 131.79156477,
       190.05585303, 136.87226954, 146.8478434 , 119.61334048,
       121.44868201, 169.65554765, 145.65663284, 160.49397597,
       145.35285195, 139.99687824, 152.56098406, 115.63296126,
       179.2578076 , 112.82372089, 129.63520802, 155.26824859,
       137.44553788, 165.30722175, 127.47522259, 129.85

In [27]:
scores

{'fit_time': array([0.00071597, 0.0006187 , 0.0006237 , 0.00060463, 0.00060773]),
 'score_time': array([0.00066829, 0.00062847, 0.00062561, 0.00062633, 0.00062919]),
 'estimator': [Lasso(), Lasso(), Lasso(), Lasso(), Lasso()],
 'test_r2': array([0.35280376, 0.37656969, 0.35072477, 0.29097008, 0.3468679 ]),
 'train_r2': array([0.36792736, 0.35382342, 0.35195324, 0.37854664, 0.34732403]),
 'test_neg_mean_squared_error': array([-3367.69481873, -3489.99166356, -3926.62532842, -4576.78631641,
        -4054.99821162]),
 'train_neg_mean_squared_error': array([-3855.43158763, -3879.43103512, -3814.09390328, -3604.01627855,
        -3819.89586116])}

In [ ]:
Parallel(n_jobs=self.n_jobs, verbose=self.verbose)(
            delayed(_fit_binary)(
                self.estimator,
                X,
                column,
                classes=[
                    "not %s" % self.label_binarizer_.classes_[i],
                    self.label_binarizer_.classes_[i],
                ],
            )
            for i, column in enumerate(columns)
        )

In [18]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
scoring = {'accuracy': make_scorer(accuracy_score),
           'prec': 'precision', "optimizing_metric": "roc_auc"}

from sklearn.model_selection import cross_validate
from sklearn.metrics import confusion_matrix
# A sample toy binary classification dataset
X, y = datasets.make_classification(n_classes=2, random_state=0)
svm = LogisticRegression(random_state=0)
svm.fit(X, y)

cv_results = cross_validate(svm, X, y, cv=5,
                            scoring=scoring)


In [8]:
cv_results

{'fit_time': array([0.0038743 , 0.00544667, 0.00362754, 0.00427985, 0.00380373]),
 'score_time': array([0.729918  , 0.00244761, 0.00238895, 0.00236964, 0.00239539]),
 'test_accuracy': array([0.75, 0.9 , 0.85, 0.65, 0.9 ]),
 'test_prec': array([0.69230769, 0.9       , 0.88888889, 0.63636364, 1.        ]),
 'test_optimizing_metric': array([0.88, 0.95, 0.93, 0.8 , 0.94])}

In [10]:
"ada".startswith("ad")

True

In [11]:
"test_optjkl"[5:]

'optjkl'

In [23]:
from sklearn.metrics._scorer import _MultimetricScorer, _check_multimetric_scoring

In [24]:
scoring = _check_multimetric_scoring(svm, scoring)

scorer = _MultimetricScorer(**scoring)

In [25]:
scorer(svm, X, y)

{'accuracy': 0.94, 'prec': 0.94, 'optimizing_metric': 0.9872000000000001}